In [1]:
import pandas as pd
import numpy as np
import pickle

import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm, grid_search
import time
import pickle


/home/bigdata/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/bigdata/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [24]:
wf= pd.read_csv('/nas/data/Zoopla/WhenFresh/whenfresh_data_linked.csv', encoding='latin1', usecols=['udprn', 'id'])
print(list(wf))

['id', 'udprn']


In [27]:
uniq__value_list = wf.id.unique().tolist()

### Define functions to get text in right format

In [32]:
# import
porter_stemmer = PorterStemmer()
wnl = WordNetLemmatizer()
from bs4 import BeautifulSoup
# 20 x  faster  for this little code here
cachedStopWords = stopwords.words("english")


def removeStopWords(input):
    exclude = set(string.punctuation)
    output = ' '.join([word for word in input.split() if word not in cachedStopWords])
    output = ''.join(ch for ch in output if ch not in exclude)
        
    return pd.Series(dict(output=output))

  
def remove_non_ascii (text):
    return ''.join(i for i in text if ord(i)<128)

def remove_non_ascii_df (text):
    return pd.Series(dict(output = ''.join(i for i in text if ord(i)<128)))

def stripHTML(input):
    output = BeautifulSoup(input, "lxml").text
    return pd.Series(dict(output = output))



### String cleaning

In [ ]:

%time ubdc['description']  = ubdc['description'] .astype(str).apply(lambda x : remove_non_ascii(x))
%time ubdc['description']  = ubdc['description'] .apply(lambda x : removeStopWords(x))
%time ubdc['description']  = ubdc['description'] .apply(stripHTML)
%time ubdc['description']  = ubdc['description'] .apply(lambda x: ' '.join([porter_stemmer.stem(y) for y in x.split()]))

ubdc.to_csv('/nas/projects/KB_proj/Z_D/UBDC_UPRN_TEXT_CLEANING/UBDC_textready.csv', index=False)


In [10]:
%time test1['description']  = test1['description'] .astype(str).apply(lambda x : remove_non_ascii(x))

CPU times: user 33.4 s, sys: 81.8 ms, total: 33.5 s
Wall time: 33.7 s


In [19]:
test= pd.read_csv('/nas/projects/KB_proj/Z_D/UBDC_UPRN_TEXT_CLEANING/UBDC_textready.csv', encoding='latin1', nrows=100)
list(test)

['unique_id', 'UPRN', 'LATITUDE', 'LONGITUDE', 'stem_text']

In [5]:
len(test)

400000

In [36]:

chunksize = 5000
chunks = 0
start = time.time() 

'''test1 = pd.read_csv('/nas/projects/KB_proj/Z_D/UBDC_UPRN_TEXT_CLEANING/UBDC_not_wf.csv', nrows = 100000, encoding='latin1', usecols = ['description', 'listing_id','unique_id','UPRN', 'LATITUDE', 'LONGITUDE'])
test1.to_csv('/nas/data/Zoopla/UBDC Zoopla Data/UBDC_UPRN/test1.csv')'''
file = '/nas/projects/KB_proj/Z_D/UBDC_UPRN_TEXT_CLEANING/UBDC_not_wf.csv'


# UBDC = "/nas/data/Zoopla/UBDC Zoopla Data/UBDC_UPRN/UBDC_UPRN2.csv" pd.read_csv(UBDC, encoding='latin1',chunksize=chunksize, usecols=cols) cols = ['description', 'listing_id','unique_id','UPRN', 'LATITUDE', 'LONGITUDE']
# Ensures that only the header for the first chunk is kept (as all the files will be appended)
writeHeader = True
# START: For each chunk in zoopla data:        
for chunk in pd.read_csv(file, chunksize=chunksize):
    #increment count
    chunks +=1
    
    %time chunk['description']  = chunk['description'].astype(str).apply(lambda x : remove_non_ascii(x))
    %time chunk['description']  = chunk['description'].apply(lambda x : removeStopWords(x))
    %time chunk['description']  = chunk['description'].apply(stripHTML)
    %time chunk['description']  = chunk['description'].apply(lambda x: ' '.join([porter_stemmer.stem(y) for y in x.split()]))

    print("chunk # ",chunks," minutes past since starting: ",(time.time() - start)//60)
        
    # 3. Write to csv and tell whether or not to include the header
    if writeHeader is True:    
        chunk.to_csv('/nas/projects/KB_proj/Z_D/UBDC_UPRN_TEXT_CLEANING/UBDC_clean.csv', mode='a', header=True, index=False)
        writeHeader = False
    else:
        chunk.to_csv('/nas/projects/KB_proj/Z_D/UBDC_UPRN_TEXT_CLEANING/UBDC_clean.csv', mode='a', header=False, index=False) 

        

CPU times: user 1.13 s, sys: 372 µs, total: 1.13 s
Wall time: 1.21 s
CPU times: user 6.07 s, sys: 9.54 ms, total: 6.08 s
Wall time: 6.1 s
CPU times: user 4.58 s, sys: 4.98 ms, total: 4.58 s
Wall time: 4.59 s
CPU times: user 22.7 s, sys: 9.31 ms, total: 22.7 s
Wall time: 22.8 s
chunk #  1  minutes past since starting:  0.0
CPU times: user 1.21 s, sys: 3.94 ms, total: 1.21 s
Wall time: 1.23 s
CPU times: user 6.37 s, sys: 4.88 ms, total: 6.38 s
Wall time: 6.4 s
CPU times: user 4.65 s, sys: 8.22 ms, total: 4.66 s
Wall time: 4.69 s
CPU times: user 24.8 s, sys: 12.4 ms, total: 24.8 s
Wall time: 25 s
chunk #  2  minutes past since starting:  1.0
CPU times: user 1.92 s, sys: 420 µs, total: 1.92 s
Wall time: 1.92 s
CPU times: user 8.74 s, sys: 13.8 ms, total: 8.76 s
Wall time: 8.81 s
CPU times: user 4.7 s, sys: 890 µs, total: 4.7 s
Wall time: 4.71 s
CPU times: user 39.8 s, sys: 19.3 ms, total: 39.8 s
Wall time: 39.9 s
chunk #  3  minutes past since starting:  2.0
CPU times: user 1.97 s, sys: 57

CPU times: user 4.64 s, sys: 214 µs, total: 4.64 s
Wall time: 4.64 s
CPU times: user 32.7 s, sys: 5.66 ms, total: 32.7 s
Wall time: 32.8 s
chunk #  26  minutes past since starting:  21.0
CPU times: user 1.64 s, sys: 3.97 ms, total: 1.64 s
Wall time: 1.65 s
CPU times: user 7.89 s, sys: 378 µs, total: 7.89 s
Wall time: 7.89 s
CPU times: user 4.99 s, sys: 253 µs, total: 4.99 s
Wall time: 5.02 s
CPU times: user 34.7 s, sys: 5.55 ms, total: 34.7 s
Wall time: 34.7 s
chunk #  27  minutes past since starting:  22.0
CPU times: user 1.75 s, sys: 89 µs, total: 1.75 s
Wall time: 1.75 s
CPU times: user 8.18 s, sys: 369 µs, total: 8.18 s
Wall time: 8.19 s
CPU times: user 4.68 s, sys: 12.2 ms, total: 4.69 s
Wall time: 4.69 s
CPU times: user 36.9 s, sys: 5.71 ms, total: 36.9 s
Wall time: 36.9 s
chunk #  28  minutes past since starting:  23.0
CPU times: user 1.32 s, sys: 0 ns, total: 1.32 s
Wall time: 1.32 s
CPU times: user 6.74 s, sys: 282 µs, total: 6.74 s
Wall time: 6.74 s
CPU times: user 4.64 s, sy

CPU times: user 1.69 s, sys: 2 µs, total: 1.69 s
Wall time: 1.69 s
CPU times: user 8.06 s, sys: 138 µs, total: 8.06 s
Wall time: 8.06 s
CPU times: user 5 s, sys: 24.1 ms, total: 5.03 s
Wall time: 5.03 s
CPU times: user 36 s, sys: 8.66 ms, total: 36 s
Wall time: 36 s
chunk #  52  minutes past since starting:  43.0
CPU times: user 1.31 s, sys: 0 ns, total: 1.31 s
Wall time: 1.31 s
CPU times: user 7.22 s, sys: 4.81 ms, total: 7.23 s
Wall time: 7.23 s
CPU times: user 4.62 s, sys: 4.08 ms, total: 4.62 s
Wall time: 4.62 s
CPU times: user 28.2 s, sys: 522 µs, total: 28.2 s
Wall time: 28.2 s
chunk #  53  minutes past since starting:  43.0
CPU times: user 1.61 s, sys: 95 µs, total: 1.61 s
Wall time: 1.61 s
CPU times: user 6.87 s, sys: 156 µs, total: 6.87 s
Wall time: 6.87 s
CPU times: user 4.58 s, sys: 4.08 ms, total: 4.59 s
Wall time: 4.59 s
CPU times: user 24.5 s, sys: 488 µs, total: 24.5 s
Wall time: 24.5 s
chunk #  54  minutes past since starting:  44.0
CPU times: user 1.14 s, sys: 0 ns, to

CPU times: user 4.69 s, sys: 4.06 ms, total: 4.7 s
Wall time: 4.7 s
CPU times: user 44.9 s, sys: 525 µs, total: 44.9 s
Wall time: 44.9 s
chunk #  77  minutes past since starting:  64.0
CPU times: user 1.9 s, sys: 0 ns, total: 1.9 s
Wall time: 1.9 s
CPU times: user 8.8 s, sys: 86 µs, total: 8.8 s
Wall time: 8.8 s
CPU times: user 4.82 s, sys: 47 µs, total: 4.82 s
Wall time: 4.82 s
CPU times: user 40.7 s, sys: 429 µs, total: 40.7 s
Wall time: 40.7 s
chunk #  78  minutes past since starting:  65.0
CPU times: user 1.93 s, sys: 0 ns, total: 1.93 s
Wall time: 1.93 s
CPU times: user 8.9 s, sys: 109 µs, total: 8.9 s
Wall time: 8.9 s
CPU times: user 4.7 s, sys: 12 ms, total: 4.71 s
Wall time: 4.71 s
CPU times: user 42.3 s, sys: 426 µs, total: 42.3 s
Wall time: 42.3 s
chunk #  79  minutes past since starting:  66.0
CPU times: user 1.61 s, sys: 35 µs, total: 1.61 s
Wall time: 1.61 s
CPU times: user 7.75 s, sys: 72 µs, total: 7.75 s
Wall time: 7.75 s
CPU times: user 4.64 s, sys: 12 ms, total: 4.65 

CPU times: user 9.19 s, sys: 7 µs, total: 9.19 s
Wall time: 9.2 s
CPU times: user 4.68 s, sys: 7.99 ms, total: 4.69 s
Wall time: 4.7 s
CPU times: user 43.5 s, sys: 20.2 ms, total: 43.5 s
Wall time: 43.6 s
chunk #  103  minutes past since starting:  86.0
CPU times: user 2.2 s, sys: 3.99 ms, total: 2.21 s
Wall time: 2.21 s
CPU times: user 9.29 s, sys: 3.99 ms, total: 9.3 s
Wall time: 9.31 s
CPU times: user 4.7 s, sys: 8.01 ms, total: 4.7 s
Wall time: 4.71 s
CPU times: user 43.7 s, sys: 4.33 ms, total: 43.7 s
Wall time: 43.7 s
chunk #  104  minutes past since starting:  87.0
CPU times: user 2.12 s, sys: 5 µs, total: 2.12 s
Wall time: 2.12 s
CPU times: user 10.1 s, sys: 96 µs, total: 10.1 s
Wall time: 10.1 s
CPU times: user 4.71 s, sys: 8.03 ms, total: 4.72 s
Wall time: 4.72 s
CPU times: user 46.5 s, sys: 4.27 ms, total: 46.5 s
Wall time: 46.5 s
chunk #  105  minutes past since starting:  88.0
CPU times: user 2.82 s, sys: 21 µs, total: 2.82 s
Wall time: 2.83 s
CPU times: user 12 s, sys: 3.

...

CPU times: user 2.11 s, sys: 4 µs, total: 2.11 s
Wall time: 2.12 s
CPU times: user 9.98 s, sys: 12 ms, total: 9.99 s
Wall time: 10 s
CPU times: user 4.7 s, sys: 12 ms, total: 4.71 s
Wall time: 4.73 s
CPU times: user 44.1 s, sys: 40.1 ms, total: 44.1 s
Wall time: 44.3 s
chunk #  2075  minutes past since starting:  1638.0
CPU times: user 1.87 s, sys: 0 ns, total: 1.87 s
Wall time: 1.87 s
CPU times: user 8.9 s, sys: 8 ms, total: 8.91 s
Wall time: 8.94 s
CPU times: user 4.68 s, sys: 8 ms, total: 4.69 s
Wall time: 4.71 s
CPU times: user 38.9 s, sys: 8.04 ms, total: 38.9 s
Wall time: 39.1 s
chunk #  2076  minutes past since starting:  1639.0
CPU times: user 1.99 s, sys: 1e+03 ns, total: 1.99 s
Wall time: 2 s
CPU times: user 9.08 s, sys: 4 ms, total: 9.09 s
Wall time: 9.12 s
CPU times: user 5.05 s, sys: 0 ns, total: 5.05 s
Wall time: 5.09 s
CPU times: user 41.5 s, sys: 28 ms, total: 41.5 s
Wall time: 41.6 s
chunk #  2077  minutes past since starting:  1640.0
CPU times: user 1.96 s, sys: 4 µs,

CPU times: user 38.2 s, sys: 32 µs, total: 38.2 s
Wall time: 38.2 s
chunk #  2100  minutes past since starting:  1662.0
CPU times: user 1.75 s, sys: 2 µs, total: 1.75 s
Wall time: 1.75 s
CPU times: user 8.36 s, sys: 9 µs, total: 8.36 s
Wall time: 8.36 s
CPU times: user 4.64 s, sys: 8 ms, total: 4.65 s
Wall time: 4.65 s
CPU times: user 38 s, sys: 38 µs, total: 38 s
Wall time: 38 s
chunk #  2101  minutes past since starting:  1663.0
CPU times: user 2.3 s, sys: 5 µs, total: 2.3 s
Wall time: 2.3 s
CPU times: user 10.1 s, sys: 7 µs, total: 10.1 s
Wall time: 10.1 s
CPU times: user 4.73 s, sys: 2 µs, total: 4.73 s
Wall time: 4.73 s
CPU times: user 49.3 s, sys: 38 µs, total: 49.3 s
Wall time: 49.3 s
chunk #  2102  minutes past since starting:  1664.0
CPU times: user 1.89 s, sys: 3 µs, total: 1.89 s
Wall time: 1.89 s
CPU times: user 8.8 s, sys: 4 µs, total: 8.8 s
Wall time: 8.8 s
CPU times: user 4.67 s, sys: 8 ms, total: 4.68 s
Wall time: 4.68 s
CPU times: user 40.6 s, sys: 46 µs, total: 40.6 s

CPU times: user 8.59 s, sys: 8.01 ms, total: 8.6 s
Wall time: 8.63 s
CPU times: user 4.72 s, sys: 4 ms, total: 4.73 s
Wall time: 4.75 s
CPU times: user 38.7 s, sys: 12 ms, total: 38.7 s
Wall time: 38.9 s
chunk #  2126  minutes past since starting:  1688.0
CPU times: user 2.68 s, sys: 0 ns, total: 2.68 s
Wall time: 2.71 s
CPU times: user 11.3 s, sys: 8.02 ms, total: 11.3 s
Wall time: 11.4 s
CPU times: user 4.82 s, sys: 8 ms, total: 4.83 s
Wall time: 4.84 s
CPU times: user 56.2 s, sys: 20 ms, total: 56.2 s
Wall time: 56.4 s
chunk #  2127  minutes past since starting:  1690.0
CPU times: user 1.43 s, sys: 4 ms, total: 1.43 s
Wall time: 1.44 s
CPU times: user 7.68 s, sys: 4.02 ms, total: 7.69 s
Wall time: 7.79 s
CPU times: user 4.65 s, sys: 0 ns, total: 4.65 s
Wall time: 4.69 s
CPU times: user 30.8 s, sys: 20 ms, total: 30.8 s
Wall time: 30.9 s
chunk #  2128  minutes past since starting:  1690.0
CPU times: user 2.18 s, sys: 4 ms, total: 2.18 s
Wall time: 2.21 s
CPU times: user 9.76 s, sys: 

CPU times: user 34.5 s, sys: 4.04 ms, total: 34.5 s
Wall time: 34.5 s
chunk #  2151  minutes past since starting:  1712.0
CPU times: user 2.09 s, sys: 1e+03 ns, total: 2.09 s
Wall time: 2.09 s
CPU times: user 9.84 s, sys: 1 µs, total: 9.84 s
Wall time: 9.85 s
CPU times: user 4.66 s, sys: 8 ms, total: 4.66 s
Wall time: 4.66 s
CPU times: user 45 s, sys: 4.03 ms, total: 45 s
Wall time: 45 s
chunk #  2152  minutes past since starting:  1713.0
CPU times: user 1.38 s, sys: 2 µs, total: 1.38 s
Wall time: 1.38 s
CPU times: user 7.41 s, sys: 8 µs, total: 7.41 s
Wall time: 7.41 s
CPU times: user 4.6 s, sys: 2 µs, total: 4.6 s
Wall time: 4.6 s
CPU times: user 29.6 s, sys: 28 ms, total: 29.6 s
Wall time: 29.7 s
chunk #  2153  minutes past since starting:  1714.0
CPU times: user 2.28 s, sys: 4.02 ms, total: 2.29 s
Wall time: 2.32 s
CPU times: user 8.5 s, sys: 7.99 ms, total: 8.51 s
Wall time: 8.58 s
CPU times: user 4.67 s, sys: 20 ms, total: 4.69 s
Wall time: 4.72 s
CPU times: user 38.1 s, sys: 28 

CPU times: user 1.98 s, sys: 4 ms, total: 1.99 s
Wall time: 2 s
CPU times: user 9.05 s, sys: 7.99 ms, total: 9.05 s
Wall time: 9.07 s
CPU times: user 4.67 s, sys: 4 ms, total: 4.67 s
Wall time: 4.68 s
CPU times: user 41.2 s, sys: 16 ms, total: 41.2 s
Wall time: 41.3 s
chunk #  2177  minutes past since starting:  1735.0
CPU times: user 1.62 s, sys: 0 ns, total: 1.62 s
Wall time: 1.62 s
CPU times: user 7.81 s, sys: 4 ms, total: 7.81 s
Wall time: 7.81 s
CPU times: user 4.59 s, sys: 0 ns, total: 4.59 s
Wall time: 4.59 s
CPU times: user 33.9 s, sys: 4.02 ms, total: 33.9 s
Wall time: 33.9 s
chunk #  2178  minutes past since starting:  1736.0
CPU times: user 1.65 s, sys: 0 ns, total: 1.65 s
Wall time: 1.65 s
CPU times: user 7.97 s, sys: 5 µs, total: 7.97 s
Wall time: 7.97 s
CPU times: user 4.57 s, sys: 16 ms, total: 4.59 s
Wall time: 4.59 s
CPU times: user 34.9 s, sys: 4.02 ms, total: 34.9 s
Wall time: 34.9 s
chunk #  2179  minutes past since starting:  1736.0
CPU times: user 1.89 s, sys: 0 n

In [38]:
tester = pd.read_csv('/nas/projects/KB_proj/Z_D/UBDC_UPRN_TEXT_CLEANING/UBDC_clean.csv', encoding='latin1', usecols=['unique_id'])

In [40]:
tester.shape

(10905375, 1)

### Perform machine learning

In [42]:
trigram_LR.fit(X_train, y_train)

chunksize = 5000
chunks = 0
start = time.time() 
ubdc = '/nas/projects/KB_proj/Z_D/UBDC_UPRN_TEXT_CLEANING/UBDC_clean.csv'
cols = ['description', 'unique_id','UPRN', 'LATITUDE', 'LONGITUDE']
# Ensures that only the header for the first chunk is kept (as all the files will be appended)
writeHeader = True
# START: For each chunk in zoopla data:        
for chunk in pd.read_csv(ubdc, chunksize=chunksize, usecols=cols):
    #increment count
    chunks +=1
    
    print("chunk # ",chunks," minutes past since starting: ",(time.time() - start)//60)
    
    %time X=chunk['description'].values.astype('U')
    
    %time predictions= trigram_LR.predict(X)
    %time y_pred_prob = trigram_LR.predict_proba(X)[:,1]
    %time X_LR_output = pd.DataFrame({'X' : X, 'predictions':predictions, 'pred_prob' :y_pred_prob, 'unique_id' :chunk['unique_id'],'lat':chunk['LATITUDE'], 'long':chunk['LONGITUDE']})
    #LR_output = pd.DataFrame({'X':X, 'predicted':y_pred_prob, 'listing_id':df_A['listing_id'], 'lat':df_A['lat'], 'long':df_A['long'] })
    #sout = X_LR_output.sort_values(['X', 'predictions'], ascending=[False, False])
        
    
    
    # 3. Write to csv and tell whether or not to include the header
    if writeHeader is True:    
        X_LR_output.to_csv('/nas/projects/KB_proj/Z_D/UBDC_UPRN_TEXT_CLEANING/UBDC_predictions.csv', mode='a', header=True, index=False)
        writeHeader = False
    else:
        X_LR_output.to_csv('/nas/projects/KB_proj/Z_D/UBDC_UPRN_TEXT_CLEANING/UBDC_predictions.csv', mode='a', header=False, index=False)


chunk #  1  minutes past since starting:  0.0
CPU times: user 97.3 ms, sys: 1e+03 ns, total: 97.3 ms
Wall time: 48.8 ms
CPU times: user 6.88 s, sys: 16 ms, total: 6.9 s
Wall time: 6.87 s
CPU times: user 7.22 s, sys: 4.01 ms, total: 7.22 s
Wall time: 7.22 s
CPU times: user 47.6 ms, sys: 0 ns, total: 47.6 ms
Wall time: 47.6 ms
chunk #  2  minutes past since starting:  0.0
CPU times: user 40.2 ms, sys: 0 ns, total: 40.2 ms
Wall time: 40.4 ms
CPU times: user 7.17 s, sys: 0 ns, total: 7.17 s
Wall time: 7.19 s
CPU times: user 7.17 s, sys: 2 µs, total: 7.17 s
Wall time: 7.17 s
CPU times: user 38.8 ms, sys: 0 ns, total: 38.8 ms
Wall time: 38.8 ms
chunk #  3  minutes past since starting:  0.0
CPU times: user 57.3 ms, sys: 0 ns, total: 57.3 ms
Wall time: 57.6 ms
CPU times: user 11.2 s, sys: 4.01 ms, total: 11.2 s
Wall time: 11.3 s
CPU times: user 11.1 s, sys: 0 ns, total: 11.1 s
Wall time: 11.1 s
CPU times: user 54.1 ms, sys: 0 ns, total: 54.1 ms
Wall time: 54.3 ms
chunk #  4  minutes past since

CPU times: user 9.28 s, sys: 8.02 ms, total: 9.29 s
Wall time: 9.3 s
CPU times: user 47.2 ms, sys: 0 ns, total: 47.2 ms
Wall time: 47.5 ms
chunk #  27  minutes past since starting:  8.0
CPU times: user 42 ms, sys: 1e+03 ns, total: 42 ms
Wall time: 42.1 ms
CPU times: user 10 s, sys: 8 ms, total: 10 s
Wall time: 10.1 s
CPU times: user 9.65 s, sys: 8 ms, total: 9.66 s
Wall time: 9.67 s
CPU times: user 39.3 ms, sys: 0 ns, total: 39.3 ms
Wall time: 39.4 ms
chunk #  28  minutes past since starting:  9.0
CPU times: user 42.9 ms, sys: 4 µs, total: 42.9 ms
Wall time: 43.1 ms
CPU times: user 10.4 s, sys: 7.99 ms, total: 10.4 s
Wall time: 10.4 s
CPU times: user 10.1 s, sys: 3.99 ms, total: 10.1 s
Wall time: 10.1 s
CPU times: user 40.5 ms, sys: 0 ns, total: 40.5 ms
Wall time: 40.5 ms
chunk #  29  minutes past since starting:  9.0
CPU times: user 32.2 ms, sys: 1e+03 ns, total: 32.2 ms
Wall time: 32.2 ms
CPU times: user 7.75 s, sys: 4.01 ms, total: 7.76 s
Wall time: 7.76 s
CPU times: user 7.72 s, sy

chunk #  52  minutes past since starting:  17.0
CPU times: user 50.9 ms, sys: 0 ns, total: 50.9 ms
Wall time: 51.1 ms
CPU times: user 9.92 s, sys: 4 ms, total: 9.93 s
Wall time: 9.95 s
CPU times: user 10.5 s, sys: 13 µs, total: 10.5 s
Wall time: 10.5 s
CPU times: user 47.5 ms, sys: 0 ns, total: 47.5 ms
Wall time: 47.5 ms
chunk #  53  minutes past since starting:  17.0
CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 56.1 ms
CPU times: user 8.12 s, sys: 7 µs, total: 8.12 s
Wall time: 8.15 s
CPU times: user 8.12 s, sys: 16 µs, total: 8.12 s
Wall time: 8.14 s
CPU times: user 56.7 ms, sys: 1e+03 ns, total: 56.7 ms
Wall time: 56.9 ms
chunk #  54  minutes past since starting:  17.0
CPU times: user 79.9 ms, sys: 0 ns, total: 79.9 ms
Wall time: 80.6 ms
CPU times: user 7.14 s, sys: 3.99 ms, total: 7.15 s
Wall time: 7.18 s
CPU times: user 7.13 s, sys: 4 ms, total: 7.13 s
Wall time: 7.16 s
CPU times: user 58 ms, sys: 0 ns, total: 58 ms
Wall time: 58.2 ms
chunk #  55  minutes past since s

CPU times: user 12.2 s, sys: 19 µs, total: 12.2 s
Wall time: 12.2 s
CPU times: user 11.6 s, sys: 3 µs, total: 11.6 s
Wall time: 11.6 s
CPU times: user 49.5 ms, sys: 0 ns, total: 49.5 ms
Wall time: 49.5 ms
chunk #  78  minutes past since starting:  25.0
CPU times: user 59 ms, sys: 3 µs, total: 59 ms
Wall time: 59.1 ms
CPU times: user 10.9 s, sys: 8 µs, total: 10.9 s
Wall time: 10.9 s
CPU times: user 10.9 s, sys: 7 µs, total: 10.9 s
Wall time: 10.9 s
CPU times: user 55.5 ms, sys: 0 ns, total: 55.5 ms
Wall time: 55.5 ms
chunk #  79  minutes past since starting:  26.0
CPU times: user 52.5 ms, sys: 1e+03 ns, total: 52.5 ms
Wall time: 52.5 ms
CPU times: user 11.2 s, sys: 7 µs, total: 11.2 s
Wall time: 11.2 s
CPU times: user 11.7 s, sys: 5 µs, total: 11.7 s
Wall time: 11.7 s
CPU times: user 49.5 ms, sys: 0 ns, total: 49.5 ms
Wall time: 49.5 ms
chunk #  80  minutes past since starting:  26.0
CPU times: user 60.9 ms, sys: 0 ns, total: 60.9 ms
Wall time: 61 ms
CPU times: user 9.42 s, sys: 5 µs, 

chunk #  103  minutes past since starting:  34.0
CPU times: user 79 ms, sys: 0 ns, total: 79 ms
Wall time: 79 ms
CPU times: user 12.3 s, sys: 4.01 ms, total: 12.3 s
Wall time: 12.3 s
CPU times: user 12 s, sys: 8 µs, total: 12 s
Wall time: 12 s
CPU times: user 74 ms, sys: 0 ns, total: 74 ms
Wall time: 74.1 ms
chunk #  104  minutes past since starting:  34.0
CPU times: user 42.8 ms, sys: 0 ns, total: 42.8 ms
Wall time: 42.8 ms
CPU times: user 11.8 s, sys: 5 µs, total: 11.8 s
Wall time: 11.8 s
CPU times: user 11.8 s, sys: 6 µs, total: 11.8 s
Wall time: 11.8 s
CPU times: user 39.6 ms, sys: 0 ns, total: 39.6 ms
Wall time: 39.6 ms
chunk #  105  minutes past since starting:  35.0
CPU times: user 51.2 ms, sys: 0 ns, total: 51.2 ms
Wall time: 51.3 ms
CPU times: user 12.5 s, sys: 10 µs, total: 12.5 s
Wall time: 12.5 s
CPU times: user 13 s, sys: 7.99 ms, total: 13 s
Wall time: 13 s
CPU times: user 47.8 ms, sys: 0 ns, total: 47.8 ms
Wall time: 48 ms
chunk #  106  minutes past since starting:  35.0

...

chunk #  2059  minutes past since starting:  655.0
CPU times: user 37.1 ms, sys: 0 ns, total: 37.1 ms
Wall time: 37.2 ms
CPU times: user 8.82 s, sys: 1e+03 ns, total: 8.82 s
Wall time: 8.82 s
CPU times: user 8.89 s, sys: 6 µs, total: 8.89 s
Wall time: 8.89 s
CPU times: user 34.4 ms, sys: 0 ns, total: 34.4 ms
Wall time: 34.4 ms
chunk #  2060  minutes past since starting:  655.0
CPU times: user 43.6 ms, sys: 2 µs, total: 43.6 ms
Wall time: 43.6 ms
CPU times: user 12.2 s, sys: 11 µs, total: 12.2 s
Wall time: 12.2 s
CPU times: user 12.6 s, sys: 10 µs, total: 12.6 s
Wall time: 12.6 s
CPU times: user 40.5 ms, sys: 0 ns, total: 40.5 ms
Wall time: 40.6 ms
chunk #  2061  minutes past since starting:  655.0
CPU times: user 70.9 ms, sys: 0 ns, total: 70.9 ms
Wall time: 71.1 ms
CPU times: user 9.37 s, sys: 6 µs, total: 9.37 s
Wall time: 9.37 s
CPU times: user 9.35 s, sys: 4 µs, total: 9.35 s
Wall time: 9.35 s
CPU times: user 66.6 ms, sys: 2 µs, total: 66.6 ms
Wall time: 66.6 ms
chunk #  2062  minu

CPU times: user 13.9 s, sys: 4.01 ms, total: 13.9 s
Wall time: 14 s
CPU times: user 14.4 s, sys: 8 ms, total: 14.4 s
Wall time: 14.5 s
CPU times: user 40.4 ms, sys: 0 ns, total: 40.4 ms
Wall time: 40.6 ms
chunk #  2085  minutes past since starting:  665.0
CPU times: user 45.5 ms, sys: 0 ns, total: 45.5 ms
Wall time: 45.7 ms
CPU times: user 10.1 s, sys: 12 ms, total: 10.1 s
Wall time: 10.1 s
CPU times: user 10.1 s, sys: 8.01 ms, total: 10.1 s
Wall time: 10.1 s
CPU times: user 38.5 ms, sys: 0 ns, total: 38.5 ms
Wall time: 38.6 ms
chunk #  2086  minutes past since starting:  665.0
CPU times: user 76.1 ms, sys: 1e+03 ns, total: 76.1 ms
Wall time: 76.4 ms
CPU times: user 11.3 s, sys: 4.01 ms, total: 11.3 s
Wall time: 11.3 s
CPU times: user 10.9 s, sys: 4 ms, total: 10.9 s
Wall time: 10.9 s
CPU times: user 53 ms, sys: 0 ns, total: 53 ms
Wall time: 53 ms
chunk #  2087  minutes past since starting:  666.0
CPU times: user 48.1 ms, sys: 0 ns, total: 48.1 ms
Wall time: 48.7 ms
CPU times: user 12.

chunk #  2110  minutes past since starting:  675.0
CPU times: user 53.6 ms, sys: 0 ns, total: 53.6 ms
Wall time: 53.6 ms
CPU times: user 13.1 s, sys: 2 µs, total: 13.1 s
Wall time: 13.1 s
CPU times: user 12.9 s, sys: 11 µs, total: 12.9 s
Wall time: 12.9 s
CPU times: user 41.9 ms, sys: 3 µs, total: 41.9 ms
Wall time: 41.9 ms
chunk #  2111  minutes past since starting:  675.0
CPU times: user 38.1 ms, sys: 1e+03 ns, total: 38.1 ms
Wall time: 38.1 ms
CPU times: user 11 s, sys: 6 µs, total: 11 s
Wall time: 11 s
CPU times: user 10.6 s, sys: 4 µs, total: 10.6 s
Wall time: 10.6 s
CPU times: user 36 ms, sys: 1e+03 ns, total: 36 ms
Wall time: 36 ms
chunk #  2112  minutes past since starting:  676.0
CPU times: user 54 ms, sys: 0 ns, total: 54 ms
Wall time: 54.1 ms
CPU times: user 12.7 s, sys: 9 µs, total: 12.7 s
Wall time: 12.7 s
CPU times: user 12.6 s, sys: 4.01 ms, total: 12.6 s
Wall time: 12.6 s
CPU times: user 40.7 ms, sys: 0 ns, total: 40.7 ms
Wall time: 40.7 ms
chunk #  2113  minutes past s

CPU times: user 11.6 s, sys: 8 ms, total: 11.6 s
Wall time: 11.6 s
CPU times: user 12 s, sys: 4.01 ms, total: 12 s
Wall time: 12 s
CPU times: user 50.2 ms, sys: 0 ns, total: 50.2 ms
Wall time: 50.5 ms
chunk #  2136  minutes past since starting:  685.0
CPU times: user 43.6 ms, sys: 2 µs, total: 43.6 ms
Wall time: 43.7 ms
CPU times: user 9.53 s, sys: 0 ns, total: 9.53 s
Wall time: 9.54 s
CPU times: user 9.52 s, sys: 7.98 ms, total: 9.53 s
Wall time: 9.54 s
CPU times: user 41.3 ms, sys: 0 ns, total: 41.3 ms
Wall time: 41.4 ms
chunk #  2137  minutes past since starting:  686.0
CPU times: user 69.4 ms, sys: 0 ns, total: 69.4 ms
Wall time: 69.6 ms
CPU times: user 12.4 s, sys: 4 ms, total: 12.4 s
Wall time: 12.4 s
CPU times: user 12 s, sys: 6 µs, total: 12 s
Wall time: 12 s
CPU times: user 45.8 ms, sys: 0 ns, total: 45.8 ms
Wall time: 45.8 ms
chunk #  2138  minutes past since starting:  686.0
CPU times: user 64.7 ms, sys: 2 µs, total: 64.7 ms
Wall time: 64.8 ms
CPU times: user 12.5 s, sys: 11

CPU times: user 8.87 s, sys: 8 µs, total: 8.87 s
Wall time: 8.87 s
CPU times: user 37.8 ms, sys: 0 ns, total: 37.8 ms
Wall time: 37.8 ms
chunk #  2161  minutes past since starting:  694.0
CPU times: user 43.8 ms, sys: 0 ns, total: 43.8 ms
Wall time: 43.8 ms
CPU times: user 10.2 s, sys: 4 ms, total: 10.2 s
Wall time: 10.2 s
CPU times: user 9.69 s, sys: 14 µs, total: 9.69 s
Wall time: 9.69 s
CPU times: user 41.2 ms, sys: 0 ns, total: 41.2 ms
Wall time: 41.3 ms
chunk #  2162  minutes past since starting:  695.0
CPU times: user 41 ms, sys: 0 ns, total: 41 ms
Wall time: 41.1 ms
CPU times: user 11.3 s, sys: 4.01 ms, total: 11.3 s
Wall time: 11.3 s
CPU times: user 10.8 s, sys: 1e+03 ns, total: 10.8 s
Wall time: 10.8 s
CPU times: user 39.1 ms, sys: 0 ns, total: 39.1 ms
Wall time: 39.1 ms
chunk #  2163  minutes past since starting:  695.0
CPU times: user 39.7 ms, sys: 2 µs, total: 39.7 ms
Wall time: 39.8 ms
CPU times: user 9.32 s, sys: 2 µs, total: 9.32 s
Wall time: 9.32 s
CPU times: user 9.32 

In [43]:
preds = pd.read_csv('/nas/projects/KB_proj/Z_D/UBDC_UPRN_TEXT_CLEANING/UBDC_predictions.csv', nrows=1000, encoding='latin1')

In [44]:
preds.head()

,X,lat,long,pred_prob,predictions,unique_id
0,We delight offer opportun leas well present 3 ...,NaN,NaN,0.050664,0,5b9f628e-98d6-4089-b984-9374e9e0ab19
1,set extens matur landscap ground convert detac...,57.173733,-2.571186,0.182986,0,1a31c2df-9595-41a1-9df1-d1256bf27be6
2,An atmospher extend croft elev posit panoram view,57.111322,-2.595603,0.061970,0,53f165e0-4a14-4edb-ae89-8f6e913c59e0
3,home report 350000 histor solid granit propert...,57.383415,-2.582475,0.117107,0,eae8d336-260d-4241-a76b-8df8ab885784
4,the hamlet whitehous situat approxim twenti mi...,57.224019,-2.633222,0.252958,0,e80cfa5b-98c2-40ea-b4b2-7f1b4f0cd15c
